In [38]:
import py4cytoscape as py4
import logging
logging.disable(logging.INFO)

In [ ]:
def apply_layout_to_cytoscape_graph(cy_graph, node_suids, layout, lower_bound=1, upper_bound=10000, resolution=1, buffer=10):
    #Get node info for creating shapes
    shape_info = get_shape_info(cy_graph, node_suids, layout)
    
    #Find scale at which shapely nodes don't overlap
    scale = find_scale(shape_info, lower_bound=lower_bound, upper_bound=upper_bound, resolution=resolution, buffer=buffer)
    print(scale)
    
    #Apply changes to cytoscape network
    scale_cytoscape_network(cy_graph, scale, shape_info)

In [37]:
def get_shape_info(cy_graph, node_suids_str, layout):
    node_suids_int = list(map(int, node_suids_str))
    
    node_heights = py4.style_values.get_node_height(node_names=node_suids_int, network=cy_graph)
    node_widths = py4.style_values.get_node_property(node_names=node_suids_int, visual_property='NODE_WIDTH', network=cy_graph)
    node_shapes = py4.style_values.get_node_property(node_names=node_suids_int, visual_property='NODE_SHAPE', network=cy_graph)
    
    shape_info = {}
    for index, node in enumerate(node_suids_int):
        shape_info[str(node)] = {
            'x': layout[index][0],
            'y': layout[index][1],
            'h': node_heights[node],
            'w': node_widths[node],
            's': node_shapes[node]
        }
    return shape_info

def find_scale(shape_info, lower_bound, upper_bound, resolution, buffer):
    return find_scale_helper(shape_info, lower_bound, upper_bound, lower_bound, upper_bound, resolution, buffer)
    
def find_scale_helper(shape_info, lo, hi, lower_bound, upper_bound, resolution, buffer):
    if lo >= hi:
        if hi >= upper_bound:
            overlap = has_overlap(upper_bound, shape_info, buffer)
            if overlap:
                print('Warning: Nodes in network continue to overlap. Please increase the upper bound from ' + str(upper_bound) + ' to solve this problem.')
            return upper_bound
        if hi <= lower_bound:
            return lower_bound
        scale = hi
        while scale <= upper_bound:
            overlap = has_overlap(scale, shape_info, buffer)
            if overlap:
                scale += resolution
            else:
                return scale
        overlap = has_overlap(scale, shape_info, buffer)
        if overlap:
            print('Warning: Nodes in network continue to overlap. Please increase the upper bound from ' + str(upper_bound) + ' to solve this problem.')
        return scale
    scale = round(((lo + hi) / 2) / resolution) * resolution
    overlap = has_overlap(scale, shape_info, buffer)
    if overlap:
        return find_scale_helper(shape_info, scale + resolution, hi, lower_bound, upper_bound, resolution, buffer)
    else:
        return find_scale_helper(shape_info, lo, scale - resolution, lower_bound, upper_bound, resolution, buffer)
    
def has_overlap(scale, shape_info, buffer):
    #Make shapes at new scale
    shapes = []
    for suid, info in shape_info.items():
        x = info['x'] * scale / 100
        y = info['y'] * scale / 100
        shapes.append(
            cyto_node_to_shapely_node(
                info['s'],
                info['h'],
                info['w'],
                x,
                y,
                buffer
            )
        )
    #Find whether shapes overlap
    overlap = False
    for i in range(len(shapes)):
        for j in range(i + 1, len(shapes)):
            if shapes[i].intersects(shapes[j]):
                overlap = True
                break
    return overlap

def scale_cytoscape_network(cy_graph, scale, shape_info):
    for suid, info in shape_info.items():
        py4.style_bypasses.set_node_property_bypass([int(suid)], info['x'] * scale / 100, 'NODE_X_LOCATION', network=cy_graph)
        py4.style_bypasses.set_node_property_bypass([int(suid)], info['y'] * scale / 100, 'NODE_Y_LOCATION', network=cy_graph)
    